# cancer_maintrain_ce

> Cross Entropy idea: basically have loss as rr + ce term 

In [ ]:
#| default_exp cancer_maintrain_ce

Setup: Surely there is a way to get rid of having to put this cell everywhere. hmmm.

Or we can just copy paste / delete this in and out when needed. Either way, getting close to a decent workable workflow.

In [ ]:
#| hide

def colab_is_true():

    try: 
        from google.colab import drive

        return True 
    except ModuleNotFoundError:
        return False

def setup_colab():
    import os
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    #os.system('unzip -q "/content/drive/My Drive/archive (1).zip"')
    os.system('git clone https://github.com/hamish-haggerty/cancer-proj.git')
    os.chdir('cancer-proj')
    os.system('unzip -q "/content/drive/My Drive/archive (1).zip"') #does this work?
    os.system('pip install -e .')
    os.system('pip install -qU nbdev')
    os.system('nbdev_install_quarto')

if __name__ == "__main__":
    on_colab = colab_is_true()
    if on_colab:
        setup_colab()

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from fastai.vision.all import *
from base_rbt.all import *
from cancer_proj.cancer_dataloading import *
from cancer_proj.cancer_metrics import *
from cancer_proj.cancer_maintrain import *

## Load the data

In [ ]:
#| hide

#Since we have cloned repository and cd'd into it (and the data itself is not stored in the
#repo) we need cd out of it, get the data, then cd back into the repo `cancer-proj`.
#This is a bit annoying, can maybe remove this later
if on_colab:
    #os.chdir('..') #assumes we are currently in cancer-proj directory
    train_dir = colab_train_dir
    test_dir = colab_test_dir
else:
    train_dir = local_train_dir
    test_dir = local_test_dir

#define general hps
device ='cuda' if torch.cuda.is_available() else 'cpu'
#bs=256
#bs=698
bs_tune=256
bs=2
size=128
bs_val=174

#get the data dictionary
data_dict = get_fnames_dls_dict(train_dir=train_dir,test_dir=test_dir,
                    device=device,bs_val=bs_val,bs=bs,bs_tune=bs_tune,size=size,n_in=3)

#get the dataloaders
dls_train,dls_tune,dls_valid = data_dict['dls_train'],data_dict['dls_tune'],data_dict['dls_valid']
x,y = data_dict['x'],data_dict['y']
xval,yval = data_dict['xval'],data_dict['yval']
xtune,ytune = data_dict['xtune'],data_dict['ytune']
vocab = data_dict['vocab']

#If we want to write some tests (make sure the data is same every time etc):
fnames,fnames_train,fnames_tune,fnames_valid,fnames_test = data_dict['fnames'],data_dict['fnames_train'],data_dict['fnames_tune'],data_dict['fnames_valid'],data_dict['fnames_test']

#test_eq(x.shape,xtune.shape)

# if on_colab:
#     os.chdir('cancer-proj')

## Load aug pipelines here

In [ ]:
#| hide

aug_dict = create_aug_pipelines(size=size,device=device,Augs=BYOL_Augs,TUNE_Augs=TUNE_Augs,Val_Augs=Val_Augs)
aug_pipelines = aug_dict['aug_pipelines']
aug_pipelines_tune = aug_dict['aug_pipelines_tune']
aug_pipelines_test = aug_dict['aug_pipelines_test'] 

## Optionally, display:

In [ ]:
#| hide
#show_bt_batch(dls=dls_train,aug=aug_pipelines,n_in=3)

In [ ]:
#| hide

#show_linear_batch(dls=dls_tune,n_in=3,aug=aug_pipelines_tune,n=2,print_augs=True)

In [ ]:
#| export


## main training api

In [ ]:
#| export

class CeBarlowTwinsModel(Module):
    """An encoder followed by a projector
    """
    def __init__(self,encoder,head):
        self.encoder = encoder
        self.head=head

    def forward(self,x):
        tem=self.encoder(x)
        return tem,self.head(tem)
    

In [ ]:
#| export

class BarlowTwinsCe(Callback):
    order,run_valid = 9,True
    def __init__(self, aug_pipelines,n_in, lmb=5e-3,numout=10, print_augs=False):
        self.aug1, self.aug2 = aug_pipelines
        if print_augs: print(self.aug1), print(self.aug2)
        store_attr('lmb')
        self.n_in=n_in
        self.cross_entropy = CrossEntropyLossFlat()
        self.numout=numout
        
        
    def before_fit(self): 
        self.learn.loss_func = self.lf
        #nf = self.learn.model.encoder[-1].out_features
        self.nf = 2048
        self.I = torch.eye(self.nf).to(self.dls.device)


    def before_epoch(self):
        pass
  
    def before_batch(self):
        
        #TODO: Make this nicer (possibly can load in data as TensorImage(BW) or something?)
        #This is a bit of a hack. Can make this more elegant later. But in new version of FastAI
        #seems we need to compute TensorImage(BW) here, and depends on whether color or not, i.e. n_in.
        if self.n_in == 1:

            xi,xj = self.aug1(TensorImageBW(self.x)), self.aug2(TensorImageBW(self.x))
            
            #print(xi.shape)
                                    
        elif self.n_in == 3:
            
            xi,xj = self.aug1(TensorImage(self.x)), self.aug2(TensorImage(self.x))

        self.learn.xb = (torch.cat([xi, xj]),)
        
    def lf(self,pred,*yb):
        
        I=self.I
        lmb=self.lmb
        
        
        pred,out = pred[0],pred[1] #encoder and head(encoder(.))
        y = yb[0]
  
        bs,nf = pred.size(0)//2,pred.size(1)
        
        z1, z2 = pred[:bs],pred[bs:] #so z1 is bs*projection_size, likewise for z2
        out1,out2 = out[:bs],out[bs:]

        z1norm = (z1 - z1.mean(0)) / z1.std(0, unbiased=False)
        z2norm = (z2 - z2.mean(0)) / z2.std(0, unbiased=False)

        C = (z1norm.T @ z2norm) / bs 
        cdiff = (C - I)**2
        #bt_loss = (cdiff*I + cdiff*(1-I)*lmb).sum()  #bt loss
        
        rr = (cdiff*(1-I)*lmb).sum()
        
        CE1 = self.cross_entropy(out1,y)
        CE2 = self.cross_entropy(out2,y)
        
        CE = 0.5*(CE1 + CE2)
        
        loss = (1/nf)*rr + CE
        
        return loss


    @torch.no_grad()
    def show(self, n=1): 
        bs = self.learn.x.size(0)//2
        x1,x2  = self.learn.x[:bs], self.learn.x[bs:]
        idxs = np.random.choice(range(bs),n,False)
        x1 = self.aug1.decode(x1[idxs].to('cpu').clone()).clamp(0,1)
        x2 = self.aug2.decode(x2[idxs].to('cpu').clone()).clamp(0,1)
        images = []
        for i in range(n): images += [x1[i],x2[i]]
        return show_batch(x1[0], None, images, max_n=len(images), nrows=n)

In [ ]:
#| export

def lf_ce(pred,*yb,I,lmb,t,criterion=CrossEntropyLossFlat()):
    

    pred,out = pred[0],pred[1] #encoder and head(encoder(.))
    y = yb[0]

    bs,nf = pred.size(0)//2,pred.size(1)

    z1, z2 = pred[:bs],pred[bs:] #so z1 is bs*projection_size, likewise for z2
    out1,out2 = out[:bs],out[bs:]

    z1norm = (z1 - z1.mean(0)) / z1.std(0, unbiased=False)
    z2norm = (z2 - z2.mean(0)) / z2.std(0, unbiased=False)

    C = (z1norm.T @ z2norm) / bs 
    cdiff = (C - I)**2
    #bt_loss = (cdiff*I + cdiff*(1-I)*lmb).sum()  #bt loss

    rr = (cdiff*(1-I)*lmb).sum()

    CE1 = criterion(out1,y)
    CE2 = criterion(out2,y)

    CE = 0.5*(CE1 + CE2)
    
    loss = t*rr + CE

    return loss



In [ ]:
#| export

@patch
def lf(self:BarlowTwinsCe, pred,*yb): return lf_ce(pred,*yb,I=self.I,lmb=self.lmb,t=self.t)

In [ ]:
#| export

@patch
def before_epoch(self:BarlowTwinsCe):
    
    self.head = nn.Linear(self.nf,self.numout) #reininitialise head before every epoch
    
    self.t=0.05

In [ ]:
# _,encoder = create_model('no_pretrain',device)
# head = nn.Linear(2048,9)

In [ ]:
# bt_ce_model = CeBarlowTwinsModel(encoder=encoder,head=head)


In [ ]:
#learn = Learner(dls_train,bt_ce_model,cbs=[BarlowTwinsCe(aug_pipelines,n_in=3,numout=9,lmb=1/8192,print_augs=False)])


In [ ]:
#learn.fit(1)

In [ ]:
# enc_1 = sequential(*[encoder[i] for i in range(5)])
# enc_2 = sequential(*[encoder[i] for i in range(5,len(encoder))])

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()